In [30]:
zones = geopandas.read_file('./geo_data/target_zones.geojson')

In [31]:
zones

,zone,radius,center_longitude,center_latitude,geometry
0,Mel North,7.214881,144.936615,-37.754349,"POLYGON ((144.86281 -37.72129, 145.01112 -37.7..."
1,Mel West,7.649048,144.795628,-37.806474,"POLYGON ((144.73509 -37.75821, 144.85731 -37.7..."
2,Mel East,8.183116,145.127579,-37.839817,"POLYGON ((145.05781 -37.78969, 145.20201 -37.7..."
3,Mel Center,6.357621,144.957748,-37.829833,"POLYGON ((144.87791 -37.80488, 145.03035 -37.8..."
4,Mel SouthEast,7.889970,145.090060,-37.949026,"POLYGON ((145.00288 -37.90031, 145.14571 -37.9..."
5,Syd North,4.229974,151.195738,-33.818880,"POLYGON ((151.15127 -33.79360, 151.22817 -33.7..."
6,Syd Center,4.102252,151.199405,-33.885837,"POLYGON ((151.16912 -33.85663, 151.24877 -33.8..."
7,Syd East,4.714001,151.245184,-33.932253,"POLYGON ((151.25770 -33.87459, 151.20414 -33.9..."
8,Syd West,5.916964,151.089241,-33.890537,"POLYGON ((151.05239 -33.84722, 151.15402 -33.8..."
9,Syd Parramatta,7.832545,150.981634,-33.820676,"POLYGON ((150.93292 -33.76735, 151.05651 -33.7..."


In [1]:
from TwitterSearch import *
try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['covid', 'vaccination']) # let's define all words we would like to have a look for
    tso.set_language('en') # we want to see German tweets only
    tso.set_include_entities(False) # and don't give us all those entity information
    tso.set_geocode(-37.840935, 144.946457, 100)
    tso.set_count(10)

    # it's about time to create a TwitterSearch object with our secret tokens
    ts = TwitterSearch(
        consumer_key = "Ozup2OAf8pHZhha38AELtzswf",
        consumer_secret = "T0QCvEWdUm2PakSKDsho9usdvaPWZsUB0hhB2XE1JwgwCGp7wV",
        access_token = "1252083222189498368-BZUECSVoWd6IDSWGnETwH4krVS3AHh",
        access_token_secret = "y9UNO50rhFndcMPy4S4cY5qLEcZdz6NygWyl8t0ZY7H8s"
     )

     # this is where the fun actually starts :)
    for tweet in ts.search_tweets_iterable(tso):
        print(tweet)
        print( '@%s tweeted: %s' % ( tweet['user']['screen_name'], tweet['text'] ) )

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

SSLError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/account/verify_credentials.json (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)'),))

In [29]:
import tweepy
import json
from sentiment_analysis import sentiment_analyzer
import geopandas
from crawler_keywords import words

consumer_key = "Ozup2OAf8pHZhha38AELtzswf"
consumer_secret = "T0QCvEWdUm2PakSKDsho9usdvaPWZsUB0hhB2XE1JwgwCGp7wV"
access_token = "1252083222189498368-BZUECSVoWd6IDSWGnETwH4krVS3AHh"
access_token_secret = "y9UNO50rhFndcMPy4S4cY5qLEcZdz6NygWyl8t0ZY7H8s"

zones = geopandas.read_file('./geo_data/target_zones.geojson')
zone_code = zones.loc[0,:].zone
zone_location=zones.loc[0,:].geometry.bounds
GEOBOX_MELBOURNE = [144.877548, -37.851203, 145.031356, -37.729655]  # https://boundingbox.klokantech.com/
GEOBOX_AUS = [106.8, -41.9, 154.9, -11.5]

# connect twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

# connect database
#sa = sentiment_analyzer.SentimentAnalyzer()

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        if(status.place is not None):
            print("-----------new tweet found----------")
            #print("created at:",status.created_at)
            #print(status.text)
            #print("-------Saving to the database-----------")
            if any(x in status.text for x in words.get('vaccine')):
                senti_score = sa.predict_sentiment(status.text)
                db.add_record(status._json)
            #print("-------finished-----------")
            #print("-----------------------------------------------")

    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            # returning False in on_error disconnects the stream
            return False

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth, listener=myStreamListener)
myStream.filter(locations=zone_location, languages=["en"])

Error during authentication


SSLError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/filter.json?delimited=length (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)'),))